In [1]:
# import required libriaries
from pathlib import Path
from helpers import *

In [2]:
# path to data directory
path = Path('/home/jupyter/fpl-prediction/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [4]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
72224,Yoshinori_Muto,8,4,0,Newcastle United,Manchester United,0.586850,1.615778,True,0,1920,1.0
72225,Youri_Tielemans,8,3,90,Leicester City,Liverpool,0.880017,2.295231,False,2,1920,1.0
72226,Yves_Bissouma,8,3,10,Brighton and Hove Albion,Tottenham Hotspur,0.471059,2.112900,True,1,1920,1.0
72227,Çaglar_Söyüncü,8,2,90,Leicester City,Liverpool,0.880017,2.295231,False,0,1920,1.0
72228,Ørjan_Nyland,8,1,0,Aston Villa,Norwich,0.409709,0.260841,False,0,1920,1.0


In [5]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

9

In [6]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [7]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season
0,Aaron_Connolly,9,4,45,Brighton and Hove Albion,Aston Villa,0.471059,0.409709,False,1,1920
1,Aaron_Cresswell,9,2,0,West Ham United,Everton,0.748095,1.085408,False,0,1920
2,Aaron_Lennon,9,3,25,Burnley,Leicester City,0.426334,0.880017,False,1,1920
3,Aaron_Mooy,9,3,34,Brighton and Hove Albion,Aston Villa,0.471059,0.409709,False,-2,1920
4,Aaron_Ramsdale,9,1,90,Bournemouth,Norwich,0.716992,0.260841,True,6,1920
5,Aaron_Wan-Bissaka,9,2,90,Manchester United,Liverpool,1.615778,2.295231,True,2,1920
6,Abdoulaye_Doucouré,9,3,90,Watford,Tottenham Hotspur,0.504415,2.112900,False,9,1920
7,Adama_Traoré,9,3,90,Wolverhampton Wanderers,Southampton,0.715919,0.508383,True,2,1920
8,Adam_Idah,9,4,0,Norwich,Bournemouth,0.260841,0.716992,False,0,1920
9,Adam_Lallana,9,3,19,Liverpool,Manchester United,2.295231,1.615778,False,7,1920


In [8]:
df_latest.shape

(555, 11)

In [9]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [10]:
last_play_proba.head()

,player,play_proba
0,Ahmed_El Mohamady,1.00
1,James_Chester,0.00
2,Neil_Taylor,1.00
3,Kortney_Hause,1.00
4,Jonathan_Kodjia,0.75


In [11]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [12]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
0,Aaron_Connolly,9,4,45,Brighton and Hove Albion,Aston Villa,0.471059,0.409709,False,1,1920,0.75
1,Aaron_Cresswell,9,2,0,West Ham United,Everton,0.748095,1.085408,False,0,1920,0.75
2,Aaron_Lennon,9,3,25,Burnley,Leicester City,0.426334,0.880017,False,1,1920,1.00
3,Aaron_Mooy,9,3,34,Brighton and Hove Albion,Aston Villa,0.471059,0.409709,False,-2,1920,1.00
4,Aaron_Ramsdale,9,1,90,Bournemouth,Norwich,0.716992,0.260841,True,6,1920,1.00
5,Aaron_Wan-Bissaka,9,2,90,Manchester United,Liverpool,1.615778,2.295231,True,2,1920,0.50
6,Abdoulaye_Doucouré,9,3,90,Watford,Tottenham Hotspur,0.504415,2.112900,False,9,1920,1.00
7,Adama_Traoré,9,3,90,Wolverhampton Wanderers,Southampton,0.715919,0.508383,True,2,1920,1.00
8,Adam_Idah,9,4,0,Norwich,Bournemouth,0.260841,0.716992,False,0,1920,1.00
9,Adam_Lallana,9,3,19,Liverpool,Manchester United,2.295231,1.615778,False,7,1920,1.00


In [13]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [14]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,season,play_proba
72779,Yoshinori_Muto,9,4,0,Newcastle United,Chelsea,0.586850,1.731076,False,0,1920,1.0
72780,Youri_Tielemans,9,3,90,Leicester City,Burnley,0.880017,0.426334,True,9,1920,1.0
72781,Yves_Bissouma,9,3,0,Brighton and Hove Albion,Aston Villa,0.471059,0.409709,False,0,1920,1.0
72782,Çaglar_Söyüncü,9,2,90,Leicester City,Burnley,0.880017,0.426334,True,2,1920,1.0
72783,Ørjan_Nyland,9,1,0,Aston Villa,Brighton and Hove Albion,0.409709,0.471059,True,0,1920,1.0


In [15]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [16]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [17]:
fixtures.head(10)

,home_team,away_team,gw
90,Arsenal,Crystal Palace,10
91,Brighton and Hove Albion,Everton,10
92,Burnley,Chelsea,10
93,Liverpool,Tottenham Hotspur,10
94,Manchester City,Aston Villa,10
95,Newcastle United,Wolverhampton Wanderers,10
96,Norwich,Manchester United,10
97,Southampton,Leicester City,10
98,Watford,Bournemouth,10
99,West Ham United,Sheffield United,10


In [18]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [19]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,10,Arsenal,Crystal Palace,3,31
1,10,Brighton and Hove Albion,Everton,36,11
2,10,Burnley,Chelsea,90,8
3,10,Liverpool,Tottenham Hotspur,14,6
4,10,Manchester City,Aston Villa,43,7


In [20]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [21]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [22]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,10,Crystal Palace,Arsenal,Patrick_van Aanholt,2,56,None,False
1,10,Crystal Palace,Arsenal,James_Tomkins,2,50,100,False
2,10,Crystal Palace,Arsenal,Mamadou_Sakho,2,50,0,False
3,10,Crystal Palace,Arsenal,Joel_Ward,2,46,None,False
4,10,Crystal Palace,Arsenal,Scott_Dann,2,44,100,False


In [23]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,10,Arsenal,Crystal Palace,Shkodran_Mustafi,2,52,None,True
1,10,Arsenal,Crystal Palace,Héctor_Bellerín,2,54,100,True
2,10,Arsenal,Crystal Palace,Sead_Kolasinac,2,54,100,True
3,10,Arsenal,Crystal Palace,Ainsley_Maitland-Niles,2,49,100,True
4,10,Arsenal,Crystal Palace,Sokratis_Papastathopoulos,2,50,None,True


In [24]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 32.5,
 'Aaron_Cresswell': 65.75,
 'Aaron_Lennon': 20.0,
 'Aaron_Mooy': 53.4,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 60.0,
 'Abdoulaye_Doucouré': 90.0,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 5.2,
 'Adam_Masina': 0.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 82.2,
 'Addji Keaninkin Marc-Israel_Guehi': 0.0,
 'Adrian_Mariappa': 18.0,
 'Adrián_San Miguel del Castillo': 72.0,
 'Ahmed_El Mohamady': 9.2,
 'Ainsley_Maitland-Niles': 32.5,
 'Akin_Famewo': 0.0,
 'Albian_Ajeti': 3.2,
 'Alex_Iwobi': 73.4,
 'Alex_McCarthy': 0.0,
 'Alex_Oxlade-Chamberlain': 21.0,
 'Alexander_Tettey': 51.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 0.0,
 'Allan_Saint-Maximin': 55.333333333333336,
 'Andre_Gray': 28.6,
 'Andreas_Christensen': 32.75,
 'Andreas_Pereira': 86.6,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 0.2,
 'Andriy_Yarmolenko': 72.2,
 'Andros_Townsend': 29.6,
 'André Filipe_Tavares Gomes': 45.0,
 'Andy_Carroll': 27.0,
 'Andy_Lonergan': 0.0,
 'Angel_Gomes': 2.25,
 'Angelo_Ogb

In [25]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [26]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,10,Arsenal,Crystal Palace,Shkodran_Mustafi,2,5.2,1.0,True,1.481711,0.507418,1920,0.000000
1,10,Arsenal,Crystal Palace,Héctor_Bellerín,2,5.4,1.0,True,1.481711,0.507418,1920,0.000000
2,10,Arsenal,Crystal Palace,Sead_Kolasinac,2,5.4,1.0,True,1.481711,0.507418,1920,90.000000
3,10,Arsenal,Crystal Palace,Ainsley_Maitland-Niles,2,4.9,1.0,True,1.481711,0.507418,1920,32.500000
4,10,Arsenal,Crystal Palace,Sokratis_Papastathopoulos,2,5.0,1.0,True,1.481711,0.507418,1920,90.000000
5,10,Arsenal,Crystal Palace,Nacho_Monreal,2,5.0,0.0,True,1.481711,0.507418,1920,0.000000
6,10,Arsenal,Crystal Palace,Laurent_Koscielny,2,5.0,0.0,True,1.481711,0.507418,1920,0.000000
7,10,Arsenal,Crystal Palace,Konstantinos_Mavropanos,2,4.4,1.0,True,1.481711,0.507418,1920,0.000000
8,10,Arsenal,Crystal Palace,Carl_Jenkinson,2,4.5,0.0,True,1.481711,0.507418,1920,0.000000
9,10,Arsenal,Crystal Palace,Rob_Holding,2,4.5,1.0,True,1.481711,0.507418,1920,0.000000


In [27]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')